In [1]:
#Image Classifier using Amazon Rekognition Service

In [2]:
#Install python packages if not installed
#!pip install boto3
#!pip install Pillow
#!pip install cognitive_face

In [3]:
import pathlib
from PIL import Image
import glob, os
import cognitive_face as CF
import shutil

#Define all input variables 
#Input the directory where the images are stored
#Please be aware of the end slash (/)

source_dir = r'/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/'
delete_dir = source_dir+'delete/'
size = 2048,2048

SUBSCRIPTION_KEY = 'a869bc0b48f846dfaaed956d80e263a9'
BASE_URL = 'https://westeurope.api.cognitive.microsoft.com/face/v1.0'
PERSON_GROUP_ID = 'known-persons'

CF.BaseUrl.set(BASE_URL)
CF.Key.set(SUBSCRIPTION_KEY)

#Initialization of directories
if not os.path.exists(delete_dir):
    os.makedirs(delete_dir)

In [4]:
#Define important functions

#Function to return a path of .jpg/.jpeg images in the directory
def return_img_list(input_dir):
    list_of_images = []
    # define the path
    currentDirectory = pathlib.Path(input_dir)
    # define the pattern to loo
    #currentPattern = pattern
#     for currentFile in currentDirectory.glob(currentPattern):
#         list_of_images.append(currentFile)
#         #print(currentFile)
    included_extenstions = ['jpg', 'jpeg', 'JPG','JPEG']
    list_of_images = [input_dir+fn for fn in os.listdir(input_dir) 
                  if any(fn.endswith(ext) for ext in included_extenstions)]
    return list_of_images

#Function to reduce the size of the images, so that transmission and scoring can be faster.
def resize_images(img_list):
    for infile in img_list:
        file,ext = os.path.splitext(infile)
        im = Image.open(infile)
        im.thumbnail(size,Image.ANTIALIAS)
        im.save(infile)

#Function to move files to delete directory :
def move_to_delete(img_path):
    img_name_list = img_path.split('/')
    img_name_list.insert(-1,'delete')
    new_img_path = '/'.join(map(str,img_name_list)) 
    #print(new_img_path)
    shutil.move(img_url,new_img_path)

In [5]:
img_list = return_img_list(source_dir)
resize_images(img_list)
thumb_img_list = return_img_list(source_dir)
#print (thumb_img_list)

In [6]:
# CF.person_group.create(PERSON_GROUP_ID,'Known Persons')
# You can use this example JPG or replace the URL below with your own URL to a JPEG image.
# for img_url in thumb_img_list:
#     faces = CF.face.detect(img_url)
#     print(faces)

In [7]:
CF.person_group.delete(PERSON_GROUP_ID)
CF.person_group.create(PERSON_GROUP_ID,'Known Persons')

{}

In [8]:
name = "Anil Panda"
user_data = 'Person group to hold pictures of Anil '
response = CF.person.create(PERSON_GROUP_ID, name, user_data)
# Get person_id from response
person_id = response['personId']

In [9]:
# Add known_faces to known person group
known_face_dir = '/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/known_faces/'

k_img_list = return_img_list(known_face_dir)
resize_images(k_img_list)
k_thumb_img_list = return_img_list(known_face_dir)

#print(k_thumb_img_list)

for img_url in k_thumb_img_list:
    faces = CF.face.detect(img_url)
    #print(faces)
    CF.person.add_face(img_url, PERSON_GROUP_ID, person_id)
    print('Added file '+ img_url)

print (CF.person.lists(PERSON_GROUP_ID))

Added file /Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/known_faces/WhatsApp Image 2017-10-09 at 7.17.16 PM.jpg
Added file /Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/known_faces/1.panda_anil_2597.jpg
Added file /Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/known_faces/WhatsApp Image 2017-10-09 at 9.08.15 PM.jpg
Added file /Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/known_faces/1.panda_anil_2597_thumbnail.JPEG
Added file /Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/known_faces/WhatsApp Image 2017-10-09 at 7.17.17 PM.jpg
Added file /Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/known_faces/WhatsApp Image 2017-10-09 at 9.08.16 PM.jpg
[{'personId': '0bbb3cf1-5f6a-4545-b5ec-9fb64b56d944', 'persistedFaceIds': ['19259170-2ecb-4d09-a0f0-9334949e5168', '21f11c77-d064-4f63-ba3b-e17b81d2b91e', '2c541be6-31c9-4248-82fa-c6527897061c', '3467c9d6-a5ec-436a-b7fb-5c4051e2a242', '42443e78-6da6-4179-8805-5ce6e51ca0dc', 'b4ff0b98-1a99-4826-ad81-35f6e2e65f8e'], 'name': 'Anil Panda', 'user

In [10]:
CF.person_group.train(PERSON_GROUP_ID)

{}

In [11]:
response = CF.person_group.get_status(PERSON_GROUP_ID)
status = response['status']
print(status)

succeeded


In [12]:
for img_url in thumb_img_list:
    response = CF.face.detect(img_url)
    face_ids = [d['faceId'] for d in response]
    #print(face_ids)
    if len(face_ids)>0:
        identified_faces = CF.face.identify(face_ids,PERSON_GROUP_ID)
        #print(identified_faces)
        for index in range(0,len(identified_faces)):
            if len(identified_faces[index]['candidates'])>0 and \
            (identified_faces[index]['candidates'][0]['confidence'])>0.6 :
                confidence = identified_faces[index]['candidates'][0]['confidence']
                print (str(img_url)+" Confidence :"+str(confidence))
                move_to_delete(img_url)

/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/IMG_0001.jpg Confidence :0.83998
/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/IMG_9966.jpg Confidence :0.82357
/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/IMG_9972.jpg Confidence :0.84734
/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/IMG_0003.jpg Confidence :0.82212
/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/IMG_0002.jpg Confidence :0.82305
/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/IMG_9973.jpg Confidence :0.84907
/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/IMG_9967.jpg Confidence :0.82434
/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/IMG_0007.jpg Confidence :0.84448
/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/IMG_0010.jpg Confidence :0.80977
/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/IMG_9975.jpg Confidence :0.80006
/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/IMG_0076.jpg Confidence :0.8561
/Users/anilkumarpanda/Pictures/Kolkata_Feb_2018/IMG_0074.jpg Confidence :0.83784
/Users/anilkumarpanda/Picture